<a href="https://colab.research.google.com/github/haruyoshikawabe/accounting-automation-python/blob/main/%E5%8B%98%E5%AE%9A%E7%A7%91%E7%9B%AE%E5%88%A4%E5%AE%9A%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
from datetime import datetime
import difflib
import re
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from google.colab import files
import io

class AccountAssistantUI:
    def __init__(self):
        self.journal_df = None
        self.review_log = []
        self.current_candidates = []
        self.setup_ui()

    def setup_ui(self):
        # スタイル定義
        style = {'description_width': '100px'}

        # ヘッダー
        self.header = widgets.HTML(
            value='<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; margin-bottom: 20px;"><h1 style="margin: 0; font-size: 28px;">📊 勘定科目判断アシスタント</h1><p style="margin: 5px 0 0 0; opacity: 0.9;">過去仕訳から勘定科目候補を提示し、判断を支援します</p></div>'
        )

        # ファイルアップロードボタン
        self.upload_btn = widgets.Button(
            description='journal.csv アップロード',
            button_style='info',
            layout=widgets.Layout(width='300px', height='40px')
        )
        self.upload_btn.on_click(self.load_journal)

        self.upload_status = widgets.HTML(value='')

        # 取引入力フォーム
        self.date_input = widgets.Text(
            description='日付:',
            placeholder='2024-01-22',
            style=style,
            layout=widgets.Layout(width='400px')
        )

        self.counterparty_input = widgets.Text(
            description='取引先:',
            placeholder='株式会社○○',
            style=style,
            layout=widgets.Layout(width='400px')
        )

        self.description_input = widgets.Text(
            description='摘要:',
            placeholder='事務用品購入',
            style=style,
            layout=widgets.Layout(width='400px')
        )

        self.amount_input = widgets.Text(
            description='金額:',
            placeholder='5000',
            style=style,
            layout=widgets.Layout(width='400px')
        )

        # 推定ボタン
        self.estimate_btn = widgets.Button(
            description='勘定科目候補を推定',
            button_style='success',
            layout=widgets.Layout(width='300px', height='40px')
        )
        self.estimate_btn.on_click(self.estimate_accounts)

        # 候補表示エリア
        self.candidates_output = widgets.Output()

        # 科目選択ドロップダウン
        self.account_select = widgets.Dropdown(
            description='選択科目:',
            options=[],
            style=style,
            layout=widgets.Layout(width='400px')
        )

        # メモ入力
        self.memo_input = widgets.Textarea(
            description='メモ:',
            placeholder='判断理由や注意点を記録できます',
            style=style,
            layout=widgets.Layout(width='400px', height='80px')
        )

        # 保存ボタン
        self.save_btn = widgets.Button(
            description='選択した科目とメモを保存',
            button_style='primary',
            layout=widgets.Layout(width='300px', height='40px')
        )
        self.save_btn.on_click(self.save_review)

        # レビューログ表示ボタン
        self.show_log_btn = widgets.Button(
            description='レビューログ表示',
            button_style='warning',
            layout=widgets.Layout(width='200px', height='35px')
        )
        self.show_log_btn.on_click(self.show_review_log)

        # CSV出力ボタン
        self.export_btn = widgets.Button(
            description='CSV出力',
            button_style='warning',
            layout=widgets.Layout(width='200px', height='35px')
        )
        self.export_btn.on_click(self.export_review_log)

        # ステータスメッセージ
        self.status_output = widgets.Output()

    def display(self):
        display(self.header)
        display(widgets.HTML(value="<h3>1. 過去仕訳データ読込</h3>"))
        display(widgets.HBox([self.upload_btn]))
        display(self.upload_status)
        display(widgets.HTML(value="<h3>2. 取引情報入力</h3>"))
        display(widgets.VBox([
            self.date_input,
            self.counterparty_input,
            self.description_input,
            self.amount_input,
            self.estimate_btn
        ]))
        display(self.candidates_output)
        display(widgets.HTML(value="<h3>3. 科目選択と保存</h3>"))
        display(widgets.VBox([
            self.account_select,
            self.memo_input,
            self.save_btn
        ]))
        display(widgets.HTML(value="<h3>4. レビューログ管理</h3>"))
        display(widgets.HBox([self.show_log_btn, self.export_btn]))
        display(self.status_output)

    def load_journal(self, btn):
        with self.status_output:
            clear_output()
            print("journal.csv をアップロードしてください...")

        uploaded = files.upload()

        with self.status_output:
            clear_output()
            for filename in uploaded.keys():
                self.journal_df = pd.read_csv(io.BytesIO(uploaded[filename]))
                self.journal_df.columns = self.journal_df.columns.str.strip()

                count = len(self.journal_df)
                cols = ', '.join(self.journal_df.columns)
                status_html = '<div style="background: #d4edda; padding: 10px; border-radius: 5px; border-left: 4px solid #28a745; margin: 10px 0;">OK ' + str(count) + '件の過去仕訳を読み込みました<br>カラム: ' + cols + '</div>'

                self.upload_status.value = status_html
                print(str(count) + "件読込完了")
                display(self.journal_df.head())

    def extract_keywords(self, text):
        if pd.isna(text) or text == '':
            return []
        keywords = re.split(r'[\s、。・]+', str(text))
        return [k for k in keywords if len(k) > 1]

    def calculate_similarity(self, str1, str2):
        if pd.isna(str1) or pd.isna(str2):
            return 0.0
        return difflib.SequenceMatcher(None, str(str1), str(str2)).ratio()

    def estimate_accounts(self, btn):
        with self.status_output:
            clear_output()

            if self.journal_df is None:
                print("先にjournal.csvをアップロードしてください")
                return

            counterparty = self.counterparty_input.value
            description = self.description_input.value

            if not description:
                print("摘要を入力してください")
                return

            col_map = {}
            for col in self.journal_df.columns:
                col_lower = col.lower()
                if '取引先' in col or 'counterparty' in col_lower:
                    col_map['counterparty'] = col
                elif '摘要' in col or 'description' in col_lower:
                    col_map['description'] = col
                elif '勘定科目' in col or 'account' in col_lower:
                    col_map['account'] = col

            if 'account' not in col_map:
                print("CSVに勘定科目カラムが見つかりません")
                return

            scores = {}
            reasons = {}

            for idx, row in self.journal_df.iterrows():
                account = row[col_map['account']]
                if pd.isna(account) or account == '':
                    continue

                if account not in scores:
                    scores[account] = 0
                    reasons[account] = []

                reason_list = []

                if 'counterparty' in col_map and counterparty:
                    row_cp = str(row[col_map['counterparty']])
                    if row_cp.lower() == counterparty.lower():
                        scores[account] += 50
                        reason_list.append('取引先 ' + row_cp + ' で過去使用')

                if 'description' in col_map:
                    row_desc = str(row[col_map['description']])
                    input_kw = self.extract_keywords(description)
                    row_kw = self.extract_keywords(row_desc)

                    matched = [kw for kw in input_kw if any(rk in kw or kw in rk for rk in row_kw)]

                    if matched:
                        scores[account] += len(matched) * 15
                        reason_list.append('キーワード一致: ' + ', '.join(matched[:3]))

                if 'description' in col_map:
                    row_desc = str(row[col_map['description']])
                    sim = self.calculate_similarity(row_desc, description)
                    if sim > 0.3:
                        scores[account] += int(sim * 20)
                        reason_list.append('類似摘要あり (' + str(int(sim*100)) + '%)')

                scores[account] += 1

                if reason_list:
                    reasons[account].extend(reason_list)

            for account in scores.keys():
                freq = len(self.journal_df[self.journal_df[col_map['account']] == account])
                reasons[account].append('過去使用回数: ' + str(freq) + '回')

            sorted_accounts = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]

            self.current_candidates = []
            for account, score in sorted_accounts:
                self.current_candidates.append({
                    'account': account,
                    'score': score,
                    'reasons': list(set(reasons[account]))[:3]
                })

            with self.candidates_output:
                clear_output()

                if not self.current_candidates:
                    print("候補が見つかりませんでした")
                    return

                html = '<div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 15px 0;">'
                html += '<h3 style="margin-top: 0;">勘定科目候補 (Top ' + str(len(self.current_candidates)) + ')</h3>'

                for i, cand in enumerate(self.current_candidates, 1):
                    badge = " [最有力]" if i == 1 else ""
                    color = "#e3f2fd" if i == 1 else "#ffffff"

                    html += '<div style="background: ' + color + '; padding: 12px; margin: 10px 0; border-left: 4px solid #2196f3; border-radius: 4px;">'
                    html += '<div style="font-size: 18px; font-weight: bold; color: #1976d2;">'
                    html += str(i) + '. ' + cand['account'] + badge
                    html += '<span style="float: right; color: #666; font-size: 14px;">スコア: ' + str(cand['score']) + '</span>'
                    html += '</div><div style="margin-top: 8px; color: #555;"><strong>判断根拠:</strong><ul style="margin: 5px 0; padding-left: 20px;">'

                    for reason in cand['reasons']:
                        html += '<li>' + reason + '</li>'

                    html += '</ul></div></div>'

                html += '</div>'
                display(HTML(html))

            self.account_select.options = [c['account'] for c in self.current_candidates]
            if self.current_candidates:
                self.account_select.value = self.current_candidates[0]['account']

            print("推定完了！上記から科目を選択して保存してください")

    def save_review(self, btn):
        with self.status_output:
            clear_output()

            if not self.account_select.value:
                print("勘定科目を選択してください")
                return

            log_entry = {
                '日付': self.date_input.value or datetime.now().strftime('%Y-%m-%d'),
                '取引先': self.counterparty_input.value,
                '摘要': self.description_input.value,
                '金額': self.amount_input.value,
                '選択科目': self.account_select.value,
                'メモ': self.memo_input.value,
                'タイムスタンプ': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            self.review_log.append(log_entry)

            print('レビューログに保存しました (累計' + str(len(self.review_log)) + '件)')

            self.date_input.value = ''
            self.counterparty_input.value = ''
            self.description_input.value = ''
            self.amount_input.value = ''
            self.memo_input.value = ''

            with self.candidates_output:
                clear_output()

    def show_review_log(self, btn):
        with self.status_output:
            clear_output()

            if not self.review_log:
                print("まだログがありません")
                return

            df = pd.DataFrame(self.review_log)
            print('レビューログ (' + str(len(self.review_log)) + '件)')
            display(df)

    def export_review_log(self, btn):
        with self.status_output:
            clear_output()

            if not self.review_log:
                print("保存されたログがありません")
                return

            df = pd.DataFrame(self.review_log)
            csv_filename = 'review_log_' + datetime.now().strftime("%Y%m%d_%H%M%S") + '.csv'
            df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

            print(csv_filename + ' を作成しました')
            display(df)
            files.download(csv_filename)


print("勘定科目判断アシスタントを起動します...")
print("")
app = AccountAssistantUI()
app.display()

勘定科目判断アシスタントを起動します...



HTML(value='<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-r…

HTML(value='<h3>1. 過去仕訳データ読込</h3>')

HTML(value='')

HTML(value='<h3>2. 取引情報入力</h3>')

Output()

HTML(value='<h3>3. 科目選択と保存</h3>')

HTML(value='<h3>4. レビューログ管理</h3>')

Output()